In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pyodbc #pyodbc is library used to connect to MSSQL


In [2]:
from datetime import datetime,date
DW_EFF_DT=date.today()

In [3]:
data_url='https://www.mohfw.gov.in/'
html_text=requests.get(data_url).text
soup=BeautifulSoup(html_text,'html5lib')
table_content=soup.find_all('table', class_='table table-striped')

In [4]:
state_df=pd.read_html(str(table_content[:]))
state_df[0]['DW_EFF_DT']=DW_EFF_DT

In [5]:
state_df[0]

,S. No.,Name of State / UT,Total Confirmed cases*,Cured/Discharged/Migrated,Deaths**,DW_EFF_DT
0,1,Andaman and Nicobar Islands,33,33,0,2020-05-26
1,2,Andhra Pradesh,3110,1896,56,2020-05-26
2,3,Arunachal Pradesh,2,1,0,2020-05-26
3,4,Assam,526,62,4,2020-05-26
4,5,Bihar,2730,749,13,2020-05-26
5,6,Chandigarh,238,186,3,2020-05-26
6,7,Chhattisgarh,291,72,0,2020-05-26
7,8,Dadar Nagar Haveli,2,0,0,2020-05-26
8,9,Delhi,14053,6771,276,2020-05-26
9,10,Goa,67,19,0,2020-05-26


In [6]:
state_dly_stats=state_df[0][['DW_EFF_DT','Name of State / UT','Total Confirmed cases*','Cured/Discharged/Migrated']]

In [7]:
#defining the reocrds to be fetchd or removal of extra records.

state_dly_stats[:-5]
state_wise_cnt=state_dly_stats[:-6]
state_wise_cnt


,DW_EFF_DT,Name of State / UT,Total Confirmed cases*,Cured/Discharged/Migrated
0,2020-05-26,Andaman and Nicobar Islands,33,33
1,2020-05-26,Andhra Pradesh,3110,1896
2,2020-05-26,Arunachal Pradesh,2,1
3,2020-05-26,Assam,526,62
4,2020-05-26,Bihar,2730,749
5,2020-05-26,Chandigarh,238,186
6,2020-05-26,Chhattisgarh,291,72
7,2020-05-26,Dadar Nagar Haveli,2,0
8,2020-05-26,Delhi,14053,6771
9,2020-05-26,Goa,67,19


In [9]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=IDEA-PC;"
                      "Database=IDW;"
                      "Trusted_Connection=yes;")

In [10]:
# code to insert data from dataframe to MSSQL.

for indx,row in state_wise_cnt.iterrows():
    with cnxn.cursor() as crsr:
        crsr.execute("INSERT INTO IDW.dbo.COVID_DLY_T(DW_EFF_DT,STATES,TOTL_CNFM_CS_IN,CURED_DISCHG) values(?,?,?,?)",row["DW_EFF_DT"],row["Name of State / UT"],row["Total Confirmed cases*"],row["Cured/Discharged/Migrated"])

In [11]:
covid_dly_t=pd.read_sql_query("select * from IDW.dbo.COVID_DLY_T",cnxn)
#this code is to generate final excel sheet which would be ingested to tableau
covid_dly_t.to_excel('COVID-Report.xlsx',index=False)
covid_dly_t

,DW_EFF_DT,STATES,TOTL_CNFM_CS_IN,CURED_DISCHG
0,2020-03-28,Andaman and Nicobar Islands,2,0
1,2020-03-28,Andhra Pradesh,14,1
2,2020-03-28,Bihar,9,0
3,2020-03-28,Chandigarh,7,0
4,2020-03-28,Chhattisgarh,6,0
...,...,...,...,...
1845,2020-05-24,Telengana,1813,1065
1846,2020-05-24,Tripura,189,153
1847,2020-05-24,Uttarakhand,244,56
1848,2020-05-24,Uttar Pradesh,6017,3406


In [ ]:
crsr.commit()
crsr.close()
cnxn.close()